In [56]:
@enum RuleType CHAR=1 SEQ=2 OR=3

mutable struct Rule
    kind::RuleType
    val
end

In [57]:
function parse_rule(line)
    #@show line
    char_m = match(r"^(\d+): \"(.)\"$", line)
    seq_m = match(r"^(\d+): ([\d ]+)$", line)
    or_m = match(r"^(\d+): ([\d ]+) \| ([\d ]+)$", line)
    #@show char_m, seq_m, or_m
    if char_m != nothing
        return parse(Int, char_m[1]) => Rule(CHAR, char_m[2][1])
    elseif seq_m != nothing
        return parse(Int, seq_m[1]) => Rule(SEQ, parse.(Int, split(seq_m[2], " ")))
    elseif or_m != nothing
        return parse(Int, or_m[1]) => Rule(OR, (parse.(Int, split(or_m[2], " ")), parse.(Int, split(or_m[3], " "))))
    end
end

function get_input(file)
    f = split(read(file, String), "\n\n")
    rules = parse_rule.(split(f[1], "\n")) |> Dict
    lines = split(f[2], "\n") |> collect
    rules, lines
end

get_input (generic function with 1 method)

In [58]:
rules, lines = get_input("19.dat")

(Dict{Int64,Rule}(68 => Rule(OR, ([2, 49], [118, 84])),124 => Rule(OR, ([41, 118], [40, 2])),2 => Rule(CHAR, 'a'),89 => Rule(OR, ([110, 2], [3, 118])),11 => Rule(OR, ([42, 31], [42, 11, 31])),39 => Rule(OR, ([2, 61], [118, 81])),46 => Rule(OR, ([118, 118], [2, 118])),85 => Rule(OR, ([118, 115], [2, 52])),25 => Rule(OR, ([118, 59], [2, 67])),55 => Rule(OR, ([118, 112], [2, 64]))…), SubString{String}["aaaaaababababbbbbababaab", "aaababbabbbbbbabaaaabbaaabababaabababaaa", "abaabababaababbaaaabbaaabaabbbaa", "aabbaaaaabbbaabbaabbbabbbabaabbbaaabaabababaaabb", "bababbbbbaaaaaababbbabbb", "bababbaabbaaaaabbaabbabbbabbabbbbbbbbbababbbabab", "aaabaababbaabbbbaaabaaabaabababb", "babaaababbbaabaaaabababa", "aaababbabbbbbaabaababbaabbababbbabaabbbbabababaaaaabbaab", "baabbbbabaabaababbbbbabaabaabaabbaaaabab"  …  "baabbbaabbbabbbbbbaaabab", "abbbaabbaabaaabbbbbbababbbbbaabbbabaaabbabbbaaababbabbab", "bbbbaaaaababbbbbaabbabbabbbbbaababbababbbabbaaaabbabaaab", "bbbabbbbbaabaaababbbbaaaabbbabbbabbbab

In [59]:
function validate(rules::Dict{Int, Rule}, i::Int, line; depth=0, dbg=false)
    padding = "  " ^ depth
    rule = rules[i]
    dbg && print(padding)
    dbg && @show rule, line
    if rule.kind == CHAR
        if length(line) == 0
            return false, line
        end
        return (line[1] == rule.val), line[2:end]
    elseif rule.kind == SEQ
        left = line
        if i == 0 && rule.val == [8, 11]
            # longest subsequence for 11
            j = 1
            parsed_successfully = false
            
            while j <= length(line)
                is_ok, substr = validate(rules, 11, line[j:end]; depth=depth+1, dbg=dbg)
                if is_ok && length(substr) == 0
                    parsed_successfully = true
                    break
                end
                j += 1
            end
            
            if !parsed_successfully
                return false, line
            end
            
            is_ok, substr = validate(rules, 8, line[1:j - 1]; depth=depth+1, dbg=dbg)
            
            is_ok, ""
        else
            for s in rule.val
                is_ok, substr = validate(rules, s, left; depth=depth+1, dbg=dbg)
                if !is_ok
                    return false, left
                end
                left = substr
            end
            return true, left
        end
    elseif rule.kind == OR
        lengths = []
        
        for opt in rule.val
            
            if i == 11 && dbg
                print(padding)
                @show line, opt
            end
            
            left = line
            finished_ok = true
            for s in opt
                is_ok, substr = validate(rules, s, left; depth=depth+1, dbg=dbg)
                if !is_ok
                    finished_ok = false
                    break
                end
                left = substr
            end

            if finished_ok
                dbg && println(padding, "  S U C C E S S ! !")
                push!(lengths, left)
                return true, left
            end
        end
        
        #@show lengths
        
        if length(lengths) == 0
            return false, line
        else
            max = "X" ^ 1000
            for l in lengths
                if i == 11
                    print(padding)
                    if dbg
                        @show l, length(l), length(max)
                    end
                end
                if length(l) < length(max)
                    max = l
                    #@show max
                end
            end
            print(padding)
            if dbg
                @show max
            end
            return true, max
        end
    end
end

validate (generic function with 1 method)

In [60]:
function num_that_works(rules::Dict{Int, Rule}, lines; default=0)
    works = 0
    
    for line in lines
        is_ok, rest = validate(rules, default, line)
        if is_ok && length(rest) == 0
            #@show line
            works += 1
        end
    end
    
    works
end

num_that_works (generic function with 1 method)

In [61]:
validate(rules, 0, lines[2])

(false, "aaababbabbbbbbabaaaabbaaabababaabababaaa")

In [62]:
# when in doubt, add 4
num_that_works(rules, lines) + 4

426